In [10]:
import os
import csv

In [11]:
# script creates .names folder for yolo and creates .txt files for each image in the dataset in the same folder
dataset_path = 'UECFOOD100/'

In [34]:
# create .names folder
# data folder can be created manually since we need to specifiy files paths there

with open(os.path.join(dataset_path, 'category.txt'), 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader)
    with open('food.names' , 'w') as f:
        for row in reader:
            f.write(row[1])
            f.write('\n')

In [15]:
with open('food.data', 'w') as f:
    f.write("classes= 100" + '\n')
    f.write("train= data/train.txt" + '\n')
    f.write("valid= data/test.txt" + '\n')
    f.write("names= data/food.names" + '\n')
    f.write("backup = /mydrive/yolov3_food/backup/")

In [14]:
# create bounding box files for each image and create new image folder for all images
from PIL import Image

def convert_to_yolo(img_size, box):
    # credit: http://bennycheung.github.io/yolo-for-real-time-food-detection
    # img_bbox file is [0:img] [1:left X] [2:bottom Y] [3:right X] [4:top Y]
    dw = 1./img_size[0]
    dh = 1./img_size[1]
    x = (int(box[1]) + int(box[3]))/2.0
    y = (int(box[2]) + int(box[4]))/2.0
    w = abs(int(box[3]) - int(box[1]))
    h = abs(int(box[4]) - int(box[2]))
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    # Yolo bbox is center x, y and width, height
    return (x,y,w,h)


train_file = open('train.txt', 'w')
test_file = open('test.txt', 'w')
count = 1
percentage = 10 # percentage split for training and testing 

curr_id = 1
with open('food.names' , 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        img_dir = os.path.join(dataset_path, str(curr_id))
        with open(os.path.join(img_dir, 'bb_info.txt'), 'r') as f:
            bb_reader = csv.reader(f, delimiter=' ')
            next(bb_reader)
            for box_row in bb_reader:
                curr_dir = os.getcwd().split('/')[-1]
                img_filename = os.path.join(img_dir, box_row[0] + '.jpg')
                # train.txt and test.txt split made here
                # filename written to .txt is different cuz they if they way they are saved in colab, the data/ folder is the folder in darknet
                if count%percentage == 0:
                    test_file.write('data/' + curr_dir + '/' + img_filename + '\n')
                else:
                    train_file.write('data/' + curr_dir + '/' + img_filename + '\n')
                count += 1
                
                # creating each image .txt file with bounding box coordinates for yolo format
                box_file = os.path.join(img_dir, box_row[0] + '.txt')
                with open(box_file, 'w') as f_box:
                    img = Image.open(img_filename)
                    yolo_box = convert_to_yolo(img.size, box_row)
                    f_box.write(str(curr_id-1) + ' ' + ' '.join(map(str, yolo_box)) + '\n')
                    img.close()
        curr_id += 1
        
train_file.close()
test_file.close()
                